In [ ]:
def interact_with_model(initial_messages):
    history = initial_messages
    response_complete = False
    while not response_complete:
        stream = ollama.chat(
            model="model-name",
            messages=history,
            stream=True
        )

        full_response = ""

        for chunk in stream:
            if 'message' in chunk and 'content' in chunk['message']:
                response_part = chunk['message']['content']
                full_response += response_part
                sys.stdout.write(response_part)
                sys.stdout.flush()

                match = re.search(r'```Python-exe\n(.*?)```',
                                  full_response, re.DOTALL)
                if match:
                    code = match.group(1)
                    execution_result = execute_Python_code(code.strip())
                    print(f"\nExecuted Result: {execution_result}")

                    if execution_result.strip():
                        history.append(
                            {"role": "assistant", "content": full_response})
                        history.append(
                            {'role': 'user', 'content': "Executed Result: " + execution_result.strip()})
                    else:
                        history.append(
                            {"role": "assistant", "content": full_response})
                        history.append({"role": "user", "content": full_response +
                                       f"\nExecution {tool_type} is successful without outputs"})
                    break

        # If code was executed, we will contiune the loop and feed the model with executed outputs
        if match:
            continue
        else:
            print()  # Move to the next line if no code was detected and streaming finished
            history.append({'role': 'assistant', 'content': full_response})
            # Exit the while loop as normal continuation if no code block found
            response_complete = True
    return history

In [ ]:
def main():
    system_message = """system message defined above"""
    history = [{
        "role": "system",
        "content": system_message
    }]
    print("OLLAMA Chat Interface. Press CTRL+C to interrupt the response or CTRL+D to exit.")

    try:
        while True:
            user_input = input(">>> ")
            if user_input.lower() == "/exit":
                print("Exiting chat.")
                break

            history.append({"role": "user", "content": user_input})
            
            try:
                # Process interaction with model including execution of code blocks
                history = interact_with_model(history)

            except KeyboardInterrupt:
                print("\nResponse interrupted by user.")
                history.append({'role': 'assistant', 'content': '[Interrupted by user]'})
                print()  # Ensure the next user prompt appears on a new line

    except EOFError:
        print("\nChat terminated via CTRL+D.")

if __name__ == "__main__":
    main()

: 